In [4]:
# Train/Test RNN
import h5py
import numpy as np
import pickle
from inputs import Data, load_separators, _build_bin_output, build_IO_from_var, build_DTA_from_var
from RNN import modelRNN
from features import get_init_end_dates, get_group_name

dateTest = ([                                                       '2018.03.09',
                '2018.03.12','2018.03.13','2018.03.14','2018.03.15','2018.03.16',
                '2018.03.19','2018.03.20','2018.03.21','2018.03.22','2018.03.23',
                '2018.03.26','2018.03.27','2018.03.28','2018.03.29','2018.03.30',
                '2018.04.02','2018.04.03','2018.04.04','2018.04.05','2018.04.06',
                '2018.04.09','2018.04.10','2018.04.11','2018.04.12','2018.04.13',
                '2018.04.16','2018.04.17','2018.04.18','2018.04.19','2018.04.20',
                '2018.04.23','2018.04.24','2018.04.25','2018.04.26','2018.04.27',
                '2018.04.30','2018.05.01','2018.05.02','2018.05.03','2018.05.04',
                '2018.05.07','2018.05.08','2018.05.09','2018.05.10','2018.05.11',
                '2018.05.14','2018.05.15','2018.05.16','2018.05.17','2018.05.18',
                '2018.05.21','2018.05.22','2018.05.23','2018.05.24','2018.05.25',
                '2018.05.28','2018.05.29','2018.05.30','2018.05.31','2018.06.01',
                '2018.06.04','2018.06.05','2018.06.06','2018.06.07','2018.06.08',
                '2018.06.11','2018.06.12','2018.06.13','2018.06.14','2018.06.15',
                '2018.06.18','2018.06.19','2018.06.20','2018.06.21','2018.06.22',
                '2018.06.25','2018.06.26','2018.06.27','2018.06.28','2018.06.29',
                '2018.07.02','2018.07.03','2018.07.04','2018.07.05','2018.07.06',
                '2018.07.09','2018.07.10','2018.07.11','2018.07.12','2018.07.13',
                '2018.07.30','2018.07.31','2018.08.01','2018.08.02','2018.08.03',
                '2018.08.06','2018.08.07','2018.08.08','2018.08.09','2018.08.10']+
               ['2018.08.13','2018.08.14','2018.08.15','2018.08.16','2018.08.17',
                '2018.08.20','2018.08.21','2018.08.22','2018.08.23','2018.08.24',
                '2018.08.27','2018.08.28','2018.08.29','2018.08.30','2018.08.31',
                '2018.09.03','2018.09.04','2018.09.05','2018.09.06','2018.09.07',
                '2018.09.10','2018.09.11','2018.09.12','2018.09.13','2018.09.14',
                '2018.09.17','2018.09.18','2018.09.19','2018.09.20','2018.09.21',
                '2018.09.24','2018.09.25','2018.09.26','2018.09.27']+['2018.09.28',
                '2018.10.01','2018.10.02','2018.10.03','2018.10.04','2018.10.05',
                '2018.10.08','2018.10.09','2018.10.10','2018.10.11','2018.10.12',
                '2018.10.15','2018.10.16','2018.10.17','2018.10.18','2018.10.19',
                '2018.10.22','2018.10.23','2018.10.24','2018.10.25','2018.10.26',
                '2018.10.29','2018.10.30','2018.10.31','2018.11.01','2018.11.02',
                '2018.11.05','2018.11.06','2018.11.07','2018.11.08','2018.11.09'])

data=Data(movingWindow=100,
          nEventsPerStat=1000,
          lB=1300, 
          dateTest=dateTest,
          assets=[1,2,3,4,7,8,10,11,12,13,14,15,16,17,19,27,28,29,30,31,32],
          channels=[0],
          max_var=10,
          feature_keys_manual=[],
          feature_keys_tsfresh=[],#[i for i in range(37,48)]+[49,50]+[i for i in range(52,68)],
          var_feat_keys=[i for i in range(68,93)])

model=modelRNN(data,
               size_hidden_layer=100,
               L=3,
               size_output_layer=5,
               keep_prob_dropout=1,
               miniBatchSize=32,
               outputGain=.6,
               commonY=3,
               lR0=0.0001,
               num_epochs=20)

tOt = 'te' # tr->train te->test
if tOt == 'tr':
    calculate_roi = False
else:
    calculate_roi = True
if_build_IO = True
from_stats_file = False

IDweights = '000400'
IDresults = '100400'
IO_results_name = IDresults
hdf5_directory = 'D:/SDC/py/HDF5/'
feats_var_directory = hdf5_directory+'feats_var/'
IO_directory = '../RNN/IO/'
if tOt=='tr':
    filename_IO = IO_directory+'IO_'+IDweights+'.hdf5'
else:
    filename_IO = IO_directory+'IO_'+IO_results_name+'.hdf5'
separators_directory = hdf5_directory+'separators/'

# if IO structures have to be built 
if if_build_IO:
    # init dictionary containing IO structures
    IO = {}
    # open IO file for writting
    f_IO = h5py.File(filename_IO,'w')
    # init IO data sets
    X = f_IO.create_dataset('X',
                            (0, model.seq_len, model.nFeatures), 
                            maxshape=(None,model.seq_len, model.nFeatures), 
                            dtype=float)
    Y = f_IO.create_dataset('Y',
                            (0,model.seq_len,model.commonY+model.size_output_layer),
                            maxshape=(None,model.seq_len,model.commonY+
                            model.size_output_layer),
                            dtype=float)
    if calculate_roi:
        D = f_IO.create_dataset('D', (0,2),
                                    maxshape=(None,2),dtype='S19')
        B = f_IO.create_dataset('B', (0,2),
                                    maxshape=(None,2),dtype=float)
        A = f_IO.create_dataset('A', (0,2),
                                    maxshape=(None,2),dtype=float)
        IO['D'] = D
        IO['B'] = B
        IO['A'] = A
    # attributes to track asset-IO belonging
    ass_IO_ass = np.zeros((len(data.assets))).astype(int)
    # structure that tracks the number of samples per level
    totalSampsPerLevel = np.zeros((model.size_output_layer))
    # save IO structures in dictionary
    IO['X'] = X
    IO['Y'] = Y
    IO['pointer'] = 0
    
for ass_idx, ass in enumerate(data.assets):
    thisAsset = data.AllAssets[str(ass)]
    print(str(ass)+" "+thisAsset)
    #thisAsset = 'EURUSD'
    # load separators
    separators = load_separators(data, 
                                 thisAsset, 
                                 separators_directory, 
                                 tOt='tr', 
                                 from_txt=1)
    aloc = 2**20
    # index asset
    #ass_idx = 0

    filename_features = (feats_var_directory+thisAsset+'_feats_var_mW'+str(data.movingWindow)+'_nE'+
                                str(data.nEventsPerStat)+'.hdf5')
    file_features = h5py.File(filename_features,'r')
    filename_returns = (feats_var_directory+thisAsset+'_rets_var_mW'+str(data.movingWindow)+'_nE'+
                                str(data.nEventsPerStat)+'.hdf5')
    file_returns = h5py.File(filename_returns,'r')
    filename_symbols = (feats_var_directory+thisAsset+'_symbols_mW'+str(data.movingWindow)+'_nE'+
                                str(data.nEventsPerStat)+'.hdf5')
    file_symbols = h5py.File(filename_symbols,'r')
    filename_stats = (feats_var_directory+thisAsset+'_stats_mW'+str(data.movingWindow)+'_nE'+
                                    str(data.nEventsPerStat)+'.p')
    # init or load total stats
    if not from_stats_file:
        stats = {}
        # load stats in
        stats["means_in"] = file_features[thisAsset].attrs.get("means_in")
        stats["stds_in"] = file_features[thisAsset].attrs.get("stds_in")
        stats["m_in"] = file_features[thisAsset].attrs.get("m_in")
        # load stats out
        stats["means_out"] = file_returns[thisAsset].attrs.get("means_out")
        stats["stds_out"] = file_returns[thisAsset].attrs.get("stds_out")
        stats["m_out"] = file_returns[thisAsset].attrs.get("m_out")
    elif from_stats_file:
        stats = pickle.load( open( filename_stats, "rb" ))
    print(stats)

    nExS = data.nEventsPerStat
    mW = data.movingWindow
    if if_build_IO:
        for s in range(0,len(separators)-1,2):#
            print("\ts {0:d} of {1:d}".format(int(s/2),int(len(separators)/2-1))+
                  ". From "+separators.DateTime.iloc[s]+" to "+
                  separators.DateTime.iloc[s+1])

            # number of events within this separator chunk
            nE = separators.index[s+1]-separators.index[s]+1
            # get first day after separator
            day_s = separators.DateTime.iloc[s][0:10]
            # check if number of events is not enough to build two features and one return
            if nE-nExS>=2*nExS:
                if (tOt == 'tr' and (day_s not in data.dateTest and day_s<=data.dateTest[-1])) or \
                    (tOt == 'te' and (day_s in data.dateTest and day_s<=data.dateTest[-1])):
                    # init and end dates
                    init_date, end_date = get_init_end_dates(separators, s)
                    #print("init_date")
                    #print(init_date)
                    #print("end_date")
                    #print(end_date)
                    # get group name
                    group_name = get_group_name(thisAsset, init_date, end_date)
                    #print("group_name")
                    #print(group_name)
                    # load features
                    if group_name in file_features:
                        features = file_features[group_name]["features"]
                    else:
                        raise ValueError(group_name+" not in "+filename_features)
                    #print("features")
                    #print(features)
                    # load returns
                    if group_name in file_returns:
                        returns = file_returns[group_name]["returns"]
                    else:
                        raise ValueError(group_name+" not in "+filename_features)
                    #print(returns)
                    # load Symbols if calculate_roi is true
                    #TODO: Implement Symbol loading for test
                    if calculate_roi:
                        DT = file_symbols[group_name]["DT"]
                        B = file_symbols[group_name]["B"]
                        A = file_symbols[group_name]["A"]
                        # build dict w. symbols
                        symbols = {'DT':DT,
                                  'B':B,
                                  'A':A}
                    else:
                        symbols = {}
                    # build IO
                    IO, totalSampsPerLevel = build_IO_from_var_notebook(data, model, stats, IO, totalSampsPerLevel, 
                                                                        features, returns, symbols, calculate_roi)
                    print(totalSampsPerLevel)
                else:
                    print("\tNot in the set. Skipped.")

            else:
                print("\ts {0:d} of {1:d}. Not enough entries. Skipped.".format(
                    int(s/2),int(len(separators)/2-1)))
        # end of for s in range(0,len(separators)-1,2):
    if if_build_IO:
        ass_IO_ass[ass_idx] = IO['pointer']
        #print("\tTime for "+thisAsset+":"+str(np.floor(time.time()-tic))+"s"+
        #          ". Total time:"+str(np.floor(time.time()-ticTotal))+"s")
# end of for ass_idx, ass in enumerate(data.assets):
if if_build_IO:
    print("Samples to RNN: "+str(max(ass_IO_ass))+".\nPercent per level:"+
            str(totalSampsPerLevel/max(ass_IO_ass)))
    f_IO.attrs.create('ass_IO_ass', ass_IO_ass, dtype=int)
    
    
    if calculate_roi:
        print("Building DTA...")
        DTA = build_DTA_from_var(data, IO['D'], IO['B'], IO['A'], ass_IO_ass)
        pickle.dump( DTA, open( "../RNN/IO/DTA"+"_"+IO_results_name+".p", "wb" ))
        f_IO.attrs.create('ass_IO_ass', ass_IO_ass, dtype=int)
        
    f_IO.close()
else:
    if calculate_roi:
        DTA = pickle.load( open( "../RNN/IO/DTA"+"_"+IO_results_name+".p", "rb" ))
    # get ass_IO_ass from disk
    f_IO = h5py.File(filename_IO,'r')
    ass_IO_ass = f_IO.attrs.get("ass_IO_ass")
    f_IO.close()
m_t = max(ass_IO_ass)
print("DONE")

1 AUDCAD
{'means_in': array([[-4.36942530e-07, -4.47432327e-07, -4.37541144e-07,
        -4.21726903e-07, -2.50684735e-07, -1.27332027e-08,
         1.93838145e-06,  4.45454320e-06, -6.33446959e+01,
         7.77982540e+00,  5.31584223e-04, -5.31164977e-04,
         5.96836233e-04, -5.85379787e-04,  5.58178248e-01,
         5.00040691e-04, -5.01337145e-04,  1.29525496e+01,
         2.61798703e-03,  2.31730762e-02,  2.48865723e-02,
         6.42509688e-02,  9.67864073e-02,  2.16410337e-01,
         3.83013997e-01]]), 'means_out': array([-2.95589297e-08, -7.53936874e-08, -2.17893512e-07, -4.45835679e-07]), 'm_out': 868773, 'stds_out': array([0.00016564, 0.00022689, 0.00033817, 0.0004634 ]), 'stds_in': array([[4.63388284e-04, 4.34756482e-04, 3.39125208e-04, 2.69818604e-04,
        1.35063097e-04, 1.01298943e-04, 8.97005624e-05, 1.11987161e-04,
        3.96523083e+00, 2.27026236e+00, 4.30828392e-04, 4.35330111e-04,
        3.37656915e-04, 3.05043884e-04, 2.53610161e-01, 4.41937573e-04,
   

[  1203.  29300. 153770.  28114.   1308.]
	s 27 of 31. From 2018.05.08 04:24:14 to 2018.07.13 23:59:57
features.shape
(57347, 25)
nSamps
57347
samp_remaining
57324
returns.shape
(57337, 4)
init_idx_rets
13
end_idx_rets
57337
r_support.shape
(57324,)
[  1629.  37112. 194545.  35999.   1734.]
	s 28 of 31. From 2018.07.30 00:05:00 to 2018.08.10 23:58:59
features.shape
(10447, 25)
nSamps
10447
samp_remaining
10424
returns.shape
(10437, 4)
init_idx_rets
13
end_idx_rets
10437
r_support.shape
(10424,)
[  1672.  38208. 202778.  37016.   1769.]
	s 29 of 31. From 2018.08.13 07:37:02 to 2018.08.23 12:52:28
features.shape
(10164, 25)
nSamps
10164
samp_remaining
10141
returns.shape
(10154, 4)
init_idx_rets
13
end_idx_rets
10154
r_support.shape
(10141,)
[  1682.  39078. 211068.  37962.   1794.]
	s 30 of 31. From 2018.08.23 13:14:34 to 2018.08.24 23:58:59
features.shape
(1681, 25)
nSamps
1681
samp_remaining
1658
returns.shape
(1671, 4)
init_idx_rets
13
end_idx_rets
1671
r_support.shape
(1658,)
[  169

[  3460.  74185. 398141.  72469.   3558.]
	s 22 of 28. From 2018.03.20 05:44:29 to 2018.04.12 09:26:15
features.shape
(14137, 25)
nSamps
14137
samp_remaining
14114
returns.shape
(14127, 4)
init_idx_rets
13
end_idx_rets
14127
r_support.shape
(14114,)
[  3582.  76300. 407641.  74707.   3697.]
	s 23 of 28. From 2018.04.12 09:59:09 to 2018.05.08 03:27:51
features.shape
(14893, 25)
nSamps
14893
samp_remaining
14870
returns.shape
(14883, 4)
init_idx_rets
13
end_idx_rets
14883
r_support.shape
(14870,)
[  3682.  78344. 418086.  76924.   3761.]
	s 24 of 28. From 2018.05.08 04:24:15 to 2018.07.13 23:59:04
features.shape
(34359, 25)
nSamps
34359
samp_remaining
34336
returns.shape
(34349, 4)
init_idx_rets
13
end_idx_rets
34349
r_support.shape
(34336,)
[  5023.  85595. 435569.  83873.   5073.]
	s 25 of 28. From 2018.07.30 00:05:00 to 2018.08.10 23:58:59
features.shape
(7485, 25)
nSamps
7485
samp_remaining
7462
returns.shape
(7475, 4)
init_idx_rets
13
end_idx_rets
7475
r_support.shape
(7462,)
[  511

features.shape
(11252, 25)
nSamps
11252
samp_remaining
11229
returns.shape
(11242, 4)
init_idx_rets
13
end_idx_rets
11242
r_support.shape
(11229,)
[  7349. 117914. 586667. 115374.   7499.]
	s 23 of 29. From 2018.03.20 05:44:28 to 2018.04.12 09:26:15
features.shape
(25971, 25)
nSamps
25971
samp_remaining
25948
returns.shape
(25961, 4)
init_idx_rets
13
end_idx_rets
25961
r_support.shape
(25948,)
[  7385. 120441. 607731. 117660.   7534.]
	s 24 of 29. From 2018.04.12 09:59:09 to 2018.05.08 03:27:49
features.shape
(25186, 25)
nSamps
25186
samp_remaining
25163
returns.shape
(25176, 4)
init_idx_rets
13
end_idx_rets
25176
r_support.shape
(25163,)
[  7412. 122153. 629296. 119512.   7541.]
	s 25 of 29. From 2018.05.08 04:24:15 to 2018.07.13 23:59:10
features.shape
(42925, 25)
nSamps
42925
samp_remaining
42902
returns.shape
(42915, 4)
init_idx_rets
13
end_idx_rets
42915
r_support.shape
(42902,)
[  7720. 128380. 659016. 125792.   7908.]
	s 26 of 29. From 2018.07.30 00:05:00 to 2018.08.10 23:58:59


features.shape
(14847, 25)
nSamps
14847
samp_remaining
14824
returns.shape
(14837, 4)
init_idx_rets
13
end_idx_rets
14837
r_support.shape
(14824,)
[  9596. 162780. 817298. 159140.   9888.]
	s 15 of 21. From 2018.03.20 05:44:32 to 2018.04.12 09:26:16
features.shape
(34783, 25)
nSamps
34783
samp_remaining
34760
returns.shape
(34773, 4)
init_idx_rets
13
end_idx_rets
34773
r_support.shape
(34760,)
[  9624. 165912. 845740. 162244.   9942.]
	s 16 of 21. From 2018.04.12 09:59:08 to 2018.05.08 03:27:53
features.shape
(35949, 25)
nSamps
35949
samp_remaining
35926
returns.shape
(35939, 4)
init_idx_rets
13
end_idx_rets
35939
r_support.shape
(35926,)
[  9700. 168552. 876324. 164853.   9959.]
	s 17 of 21. From 2018.05.08 04:24:12 to 2018.07.13 23:59:11
features.shape
(58847, 25)
nSamps
58847
samp_remaining
58824
returns.shape
(58837, 4)
init_idx_rets
13
end_idx_rets
58837
r_support.shape
(58824,)
[ 10017. 175738. 920513. 171663.  10281.]
	s 18 of 21. From 2018.07.30 00:05:00 to 2018.08.10 23:58:59


...

[  13069.  246350. 1450612.  240745.   13412.]
	s 14 of 20. From 2018.03.19 01:00:00 to 2018.03.20 05:04:41
features.shape
(976, 25)
nSamps
976
samp_remaining
953
returns.shape
(966, 4)
init_idx_rets
13
end_idx_rets
966
r_support.shape
(953,)
[  13071.  246495. 1451246.  240911.   13418.]
	s 15 of 20. From 2018.03.20 05:44:33 to 2018.03.23 22:58:57
features.shape
(4232, 25)
nSamps
4232
samp_remaining
4209
returns.shape
(4222, 4)
init_idx_rets
13
end_idx_rets
4222
r_support.shape
(4209,)
[  13137.  247224. 1453656.  241792.   13541.]
	s 16 of 20. From 2018.03.26 01:00:59 to 2018.03.29 23:58:56
features.shape
(4185, 25)
nSamps
4185
samp_remaining
4162
returns.shape
(4175, 4)
init_idx_rets
13
end_idx_rets
4175
r_support.shape
(4162,)
[  13184.  248044. 1456262.  242461.   13561.]
	s 17 of 20. From 2018.04.02 01:01:00 to 2018.04.12 09:26:16
features.shape
(8855, 25)
nSamps
8855
samp_remaining
8832
returns.shape
(8845, 4)
init_idx_rets
13
end_idx_rets
8845
r_support.shape
(8832,)
[  13355. 

features.shape
(5960, 25)
nSamps
5960
samp_remaining
5937
returns.shape
(5950, 4)
init_idx_rets
13
end_idx_rets
5950
r_support.shape
(5937,)
[  14961.  274112. 1572626.  268874.   15223.]
	s 19 of 25. From 2018.03.20 05:44:30 to 2018.04.12 09:26:15
features.shape
(15679, 25)
nSamps
15679
samp_remaining
15656
returns.shape
(15669, 4)
init_idx_rets
13
end_idx_rets
15669
r_support.shape
(15656,)
[  15017.  276072. 1584182.  270898.   15283.]
	s 20 of 25. From 2018.04.12 09:59:08 to 2018.05.08 03:27:52
features.shape
(17069, 25)
nSamps
17069
samp_remaining
17046
returns.shape
(17059, 4)
init_idx_rets
13
end_idx_rets
17059
r_support.shape
(17046,)
[  15055.  277957. 1597014.  273135.   15337.]
	s 21 of 25. From 2018.05.08 04:24:15 to 2018.07.13 23:59:00
features.shape
(29066, 25)
nSamps
29066
samp_remaining
29043
returns.shape
(29056, 4)
init_idx_rets
13
end_idx_rets
29056
r_support.shape
(29043,)
[  15468.  282699. 1615819.  277858.   15697.]
	s 22 of 25. From 2018.07.30 00:05:00 to 2018.0

features.shape
(8573, 25)
nSamps
8573
samp_remaining
8550
returns.shape
(8563, 4)
init_idx_rets
13
end_idx_rets
8563
r_support.shape
(8550,)
[  16355.  306684. 1764499.  302640.   16568.]
	s 16 of 22. From 2018.03.20 05:44:32 to 2018.04.12 09:26:16
features.shape
(21672, 25)
nSamps
21672
samp_remaining
21649
returns.shape
(21662, 4)
init_idx_rets
13
end_idx_rets
21662
r_support.shape
(21649,)
[  16519.  309964. 1778904.  306286.   16722.]
	s 17 of 22. From 2018.04.12 10:00:39 to 2018.05.08 03:27:53
features.shape
(18979, 25)
nSamps
18979
samp_remaining
18956
returns.shape
(18969, 4)
init_idx_rets
13
end_idx_rets
18969
r_support.shape
(18956,)
[  16571.  311918. 1793924.  308176.   16762.]
	s 18 of 22. From 2018.05.08 04:24:16 to 2018.05.31 23:58:58
features.shape
(19161, 25)
nSamps
19161
samp_remaining
19138
returns.shape
(19151, 4)
init_idx_rets
13
end_idx_rets
19151
r_support.shape
(19138,)
[  16806.  314848. 1806788.  311125.   16922.]
	s 19 of 22. From 2018.07.30 00:05:00 to 2018.0

[  19659.  359519. 2017441.  355370.   19805.]
	s 15 of 21. From 2018.03.20 05:44:29 to 2018.04.12 09:26:16
features.shape
(26447, 25)
nSamps
26447
samp_remaining
26424
returns.shape
(26437, 4)
init_idx_rets
13
end_idx_rets
26437
r_support.shape
(26424,)
[  19829.  363564. 2035367.  359460.   19998.]
	s 16 of 21. From 2018.04.12 10:00:38 to 2018.05.08 03:27:55
features.shape
(21570, 25)
nSamps
21570
samp_remaining
21547
returns.shape
(21560, 4)
init_idx_rets
13
end_idx_rets
21560
r_support.shape
(21547,)
[  19851.  365865. 2052383.  361619.   20047.]
	s 17 of 21. From 2018.05.08 04:24:13 to 2018.07.13 23:59:08
features.shape
(41673, 25)
nSamps
41673
samp_remaining
41650
returns.shape
(41663, 4)
init_idx_rets
13
end_idx_rets
41663
r_support.shape
(41650,)
[  20498.  373074. 2078142.  369039.   20662.]
	s 18 of 21. From 2018.07.30 00:05:00 to 2018.08.10 23:58:59
features.shape
(8174, 25)
nSamps
8174
samp_remaining
8151
returns.shape
(8164, 4)
init_idx_rets
13
end_idx_rets
8164
r_support.

[  22001.  408030. 2259949.  404001.   22268.]
	s 16 of 23. From 2018.03.20 05:44:29 to 2018.04.12 09:26:16
features.shape
(39550, 25)
nSamps
39550
samp_remaining
39527
returns.shape
(39540, 4)
init_idx_rets
13
end_idx_rets
39540
r_support.shape
(39527,)
[  22061.  410828. 2293685.  406875.   22327.]
	s 17 of 23. From 2018.04.12 09:59:09 to 2018.05.08 03:27:56
features.shape
(35249, 25)
nSamps
35249
samp_remaining
35226
returns.shape
(35239, 4)
init_idx_rets
13
end_idx_rets
35239
r_support.shape
(35226,)
[  22127.  412881. 2324923.  408729.   22342.]
	s 18 of 23. From 2018.05.08 04:24:15 to 2018.07.13 23:59:56
features.shape
(65322, 25)
nSamps
65322
samp_remaining
65299
returns.shape
(65312, 4)
init_idx_rets
13
end_idx_rets
65312
r_support.shape
(65299,)
[  22452.  420422. 2374600.  416129.   22698.]
	s 19 of 23. From 2018.07.30 00:05:00 to 2018.08.10 23:58:59
features.shape
(13458, 25)
nSamps
13458
samp_remaining
13435
returns.shape
(13448, 4)
init_idx_rets
13
end_idx_rets
13448
r_sup

In [8]:
m_t

733966

In [3]:
# Train
import tensorflow as tf
# reset graph
alloc = 2**20
tf.reset_default_graph()
# start session
with tf.Session() as sess:    
    model.train(sess, int(np.ceil(m_t/aloc)), ID=IDweights, IDIO=IDweights, 
                data_format='hdf5', filename_IO=filename_IO, aloc=alloc)

ID = 000400
Training from epoch 0 till 19
Time training Chunck 0 of 14: 664 s
Time training Chunck 1 of 14: 664 s
Time training Chunck 2 of 14: 664 s
Time training Chunck 3 of 14: 669 s
Time training Chunck 4 of 14: 675 s
Time training Chunck 5 of 14: 673 s
Time training Chunck 6 of 14: 675 s
Time training Chunck 7 of 14: 673 s
Time training Chunck 8 of 14: 678 s
Time training Chunck 9 of 14: 677 s
Time training Chunck 10 of 14: 678 s
Time training Chunck 11 of 14: 681 s
Time training Chunck 12 of 14: 557 s
Time training Chunck 13 of 14: 530 s
Time training Chunck 14 of 14: 354 s
Cost after epoch 0: 0.524633. Av cost 4.306789
21:32:07 Total time training: 158.63m
INFO:tensorflow:../RNN/weights/000400/000000/000000 is not in all_model_checkpoint_paths. Manually adding it.
Parameters saved
Time training Chunck 0 of 14: 386 s
Time training Chunck 1 of 14: 382 s
Time training Chunck 2 of 14: 682 s
Time training Chunck 3 of 14: 691 s
Time training Chunck 4 of 14: 691 s
Time training Chunck 

KeyboardInterrupt: 

In [8]:
# Test
import tensorflow as tf

alloc = 200000
save_journal = False
startFrom = -1
endAt = -1
# reset graph
tf.reset_default_graph()
# start session
with tf.Session() as sess:
    # run test RNN
    print("IDresults: "+IDresults)
    # TEMP: GRE calculation not implemented in test2 yet. Use old test
    if save_journal:
        model.test(sess, data, IDresults, IDweights, 
                    alloc, 1, 'test', startFrom=startFrom,
                    IDIO=IO_results_name, data_format='hdf5', DTA=DTA, 
                    save_journal=save_journal, endAt=endAt)
    else:
        model.test2(sess, data, IDresults, IDweights, 
                    alloc, 1, 'test', startFrom=startFrom,
                    IDIO=IO_results_name, data_format='hdf5', DTA=DTA, 
                    save_journal=save_journal, endAt=endAt, from_var=True)
        print("DONE")

IDresults: 1TESTVAR
INFO:tensorflow:Restoring parameters from ../RNN/weights/0TESTVAR/000006/000006
Parameters loaded. Epoch 6
Epoch 6 of 9. Getting output...
Chunck 1 of 1
Getting results
Epoch = 6. Time index = 0. Threshold MC = 0.5. Threshold MD = 0.5
J_test = 4.4921823, J_train = 4.598583055835617, Accuracy=0.6831349752083318
RD = 697 NZ = 1312 NZA = 2234 pNZ = 0.975% pNZA = 1.661% AD = 53.12% ADA = 31.20% NO = 228 GSP = 50.88% NSP = 40.35%
SI2 = -25.40 SI = -33.40 eGROI = 0.92% eROI = -5.30% eROI2 = -3.64% eROI3 = -5.92% mSpread = 0.0273%
Epoch = 6. Time index = 0. Threshold MC = 0.5. Threshold MD = 0.6
RD = 54 NZ = 92 NZA = 152 pNZ = 0.068% pNZA = 0.113% AD = 58.70% ADA = 35.53% NO = 62 GSP = 54.84% NSP = 38.71%
SI2 = -8.10 SI = -10.10 eGROI = 0.11% eROI = -1.71% eROI2 = -1.13% eROI3 = -1.75% mSpread = 0.0294%


C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\numpy\core\fromnumeric.py:3194: RuntimeWarning: Degrees of freedom <= 0 for slice
  **kwargs)
C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\numpy\core\_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
C:\ProgramData\Anaconda3\envs\py35\lib\site-packages\numpy\core\_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch = 6. Time index = 0. Threshold MC = 0.5. Threshold MD = 0.7
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%
Epoch = 6. Time index = 0. Threshold MC = 0.5. Threshold MD = 0.8
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%
Epoch = 6. Time index = 0. Threshold MC = 0.5. Threshold MD = 0.9
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%

Epoch = 6. Time index = 0. Threshold MC = 0.6. Threshold MD = 0.5
RD = 130 NZ = 275 NZA = 418 pNZ = 0.204% pNZA = 0.311% AD = 47.27% ADA = 31.10% NO = 58 GSP = 56.90% NSP = 41.38%
SI2 = -4.90 SI = -7.90 e

Epoch = 6. Time index = 1. Threshold MC = 0.6. Threshold MD = 0.8
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%
Epoch = 6. Time index = 1. Threshold MC = 0.6. Threshold MD = 0.9
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%

Epoch = 6. Time index = 1. Threshold MC = 0.7. Threshold MD = 0.5
RD = 22 NZ = 43 NZA = 61 pNZ = 0.032% pNZA = 0.045% AD = 51.16% ADA = 36.07% NO = 21 GSP = 52.38% NSP = 42.86%
SI2 = -1.55 SI = -2.55 eGROI = 0.43% eROI = -0.14% eROI2 = 0.01% eROI3 = -0.20% mSpread = 0.0272%
Epoch = 6. Time index = 1. Threshold MC = 0.7. Threshold MD = 0.6
RD = 2 NZ = 2 NZA = 4 pNZ = 0.001% pNZA = 0.003% AD = 100.00% ADA = 50.00% NO = 4 GSP = 50.00% NSP = 50.00%
SI2 = -0.20 SI = -0.

Epoch = 6. Time index = 2. Threshold MC = 0.7. Threshold MD = 0.9
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%

Epoch = 6. Time index = 2. Threshold MC = 0.8. Threshold MD = 0.5
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%
Epoch = 6. Time index = 2. Threshold MC = 0.8. Threshold MD = 0.6
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%
Epoch = 6. Time index = 2. Threshold MC = 0.8. Threshold MD = 0.7
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00

...

Epoch = 8. Time index = 3. Threshold MC = 0.5. Threshold MD = 0.6
RD = 68 NZ = 118 NZA = 207 pNZ = 0.088% pNZA = 0.154% AD = 57.63% ADA = 32.85% NO = 94 GSP = 58.51% NSP = 41.49%
SI2 = -10.70 SI = -12.70 eGROI = 1.27% eROI = -1.93% eROI2 = -0.61% eROI3 = -1.55% mSpread = 0.0340%
Epoch = 8. Time index = 3. Threshold MC = 0.5. Threshold MD = 0.7
RD = 27 NZ = 33 NZA = 59 pNZ = 0.025% pNZA = 0.044% AD = 81.82% ADA = 45.76% NO = 34 GSP = 64.71% NSP = 44.12%
SI2 = -1.70 SI = -3.70 eGROI = 0.78% eROI = -0.55% eROI2 = 0.10% eROI3 = -0.24% mSpread = 0.0392%
Epoch = 8. Time index = 3. Threshold MC = 0.5. Threshold MD = 0.8
RD = 7 NZ = 9 NZA = 16 pNZ = 0.007% pNZA = 0.012% AD = 77.78% ADA = 43.75% NO = 9 GSP = 66.67% NSP = 44.44%
SI2 = -0.95 SI = -0.95 eGROI = 0.53% eROI = 0.19% eROI2 = 0.35% eROI3 = 0.26% mSpread = 0.0369%
Epoch = 8. Time index = 3. Threshold MC = 0.5. Threshold MD = 0.9
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = 

Epoch = 9. Time index = 0. Threshold MC = 0.5. Threshold MD = 0.8
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%
Epoch = 9. Time index = 0. Threshold MC = 0.5. Threshold MD = 0.9
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%

Epoch = 9. Time index = 0. Threshold MC = 0.6. Threshold MD = 0.5
RD = 167 NZ = 311 NZA = 468 pNZ = 0.231% pNZA = 0.348% AD = 53.70% ADA = 35.68% NO = 69 GSP = 55.07% NSP = 39.13%
SI2 = -7.95 SI = -10.95 eGROI = 0.69% eROI = -1.30% eROI2 = -0.69% eROI3 = -1.38% mSpread = 0.0288%
Epoch = 9. Time index = 0. Threshold MC = 0.6. Threshold MD = 0.6
RD = 8 NZ = 19 NZA = 29 pNZ = 0.014% pNZA = 0.022% AD = 42.11% ADA = 27.59% NO = 17 GSP = 41.18% NSP = 17.65%
SI2 = -4.35 S

Epoch = 9. Time index = 1. Threshold MC = 0.6. Threshold MD = 0.9
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%

Epoch = 9. Time index = 1. Threshold MC = 0.7. Threshold MD = 0.5
RD = 37 NZ = 72 NZA = 104 pNZ = 0.054% pNZA = 0.077% AD = 51.39% ADA = 35.58% NO = 31 GSP = 58.06% NSP = 38.71%
SI2 = -3.05 SI = -5.05 eGROI = 0.20% eROI = -0.78% eROI2 = -0.42% eROI3 = -0.73% mSpread = 0.0314%
Epoch = 9. Time index = 1. Threshold MC = 0.7. Threshold MD = 0.6
RD = 6 NZ = 12 NZA = 17 pNZ = 0.009% pNZA = 0.013% AD = 50.00% ADA = 35.29% NO = 11 GSP = 45.45% NSP = 45.45%
SI2 = -1.05 SI = -1.05 eGROI = 0.26% eROI = -0.02% eROI2 = 0.04% eROI3 = -0.07% mSpread = 0.0247%
Epoch = 9. Time index = 1. Threshold MC = 0.7. Threshold MD = 0.7
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI


Epoch = 9. Time index = 2. Threshold MC = 0.8. Threshold MD = 0.5
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%
Epoch = 9. Time index = 2. Threshold MC = 0.8. Threshold MD = 0.6
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%
Epoch = 9. Time index = 2. Threshold MC = 0.8. Threshold MD = 0.7
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%
Epoch = 9. Time index = 2. Threshold MC = 0.8. Threshold MD = 0.8
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00

Epoch = 9. Time index = 3. Threshold MC = 0.9. Threshold MD = 0.6
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%
Epoch = 9. Time index = 3. Threshold MC = 0.9. Threshold MD = 0.7
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%
Epoch = 9. Time index = 3. Threshold MC = 0.9. Threshold MD = 0.8
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00% eROI = 0.00% eROI2 = 0.00% eROI3 = 0.00% mSpread = 0.0000%
Epoch = 9. Time index = 3. Threshold MC = 0.9. Threshold MD = 0.9
RD = 0 NZ = 0 NZA = 0 pNZ = 0.000% pNZA = 0.000% AD = 0.00% ADA = 0.00% NO = 1 GSP = 0.00% NSP = 0.00%
SI2 = -0.55 SI = -0.55 eGROI = 0.00%

In [1]:
# Function Build IO from var
def build_IO_from_var_notebook(data, model, stats, IO, totalSampsPerLevel, features, returns, symbols, calculate_roi):
    # total number of possible channels
    nExS = data.nEventsPerStat
    mW = data.movingWindow
    nChannels = int(nExS/mW)
    # sequence length
    seq_len = model.seq_len#int((data.lB-data.nEventsPerStat)/data.movingWindow)
    # samples allocation per batch
    aloc = 2**20
    # extract means and stats
    means_in = stats['means_in']
    stds_in = stats['stds_in']
    #m_in = stats['m_in']
    stds_out = stats['stds_out']
    #m_out = stats['m_out']
    #print("m_in")
    #print(m_in)
    #print("m_out")
    #print(m_out)
    # add dateTimes, bids and asks if are included in file
    if calculate_roi:
        dts = symbols['DT']
        bids = symbols['B']
        asks = symbols['A']

        D = IO['D']
        B = IO['B']
        A = IO['A']

    # extract IO structures
    X = IO['X']
    Y = IO['Y']
    #I = IO['I']
    pointer = IO['pointer']

    feats_var_normed = np.minimum(np.maximum((features-means_in)/\
                         stds_in,-data.max_var),data.max_var)
    # get some scalars
    nSamps = feats_var_normed.shape[0]
    print("features.shape")
    print(features.shape)
    print("nSamps")
    print(nSamps)
    samp_remaining = nSamps-2*nChannels-seq_len+1
    print("samp_remaining")
    print(samp_remaining)
    print("returns.shape")
    print(returns.shape)
    chunks = int(np.ceil(samp_remaining/aloc))
    # init counter of samps processed
    offset = 0
    # loop over chunks
    for i in range(chunks):
        # this batch length
        batch = np.min([samp_remaining, aloc])
        # create support numpy vectors to speed up iterations
        v_support = feats_var_normed[offset:offset+batch+seq_len, :]
        # get init and end indexes for returns
        init_idx_rets = nChannels+offset+seq_len-1
        print("init_idx_rets")
        print(init_idx_rets)
        end_idx_rets = nChannels+offset+batch+seq_len-1#+2*seq_len-1
        print("end_idx_rets")
        print(end_idx_rets)
        r_support = returns[init_idx_rets:end_idx_rets, data.lookAheadIndex]
        print("r_support.shape")
        print(r_support.shape)
        assert(r_support.shape[0]==end_idx_rets-init_idx_rets)
        # we only take here the entry time index, and later at DTA building the 
        # exit time index is derived from the entry time and the number of events to
        # look ahead
        if calculate_roi:
            dt_support = dts[init_idx_rets:end_idx_rets, [0,data.lookAheadIndex+1]]
            b_support = bids[init_idx_rets:end_idx_rets, [0,data.lookAheadIndex+1]]
            a_support = asks[init_idx_rets:end_idx_rets, [0,data.lookAheadIndex+1]]
        # update remaining samps to proceed
        samp_remaining = samp_remaining-batch
        # init formatted input and output
        X_i = np.zeros((batch, seq_len, features.shape[1]))
        # real-valued output
        O_i = np.zeros((batch, seq_len, 1))    
        if calculate_roi:
            # last dimension is to incorporate in and out symbols
            D_i = np.chararray((batch, 2),itemsize=19)
            B_i = np.zeros((batch, 2))
            A_i = np.zeros((batch, 2))

        for nI in range(batch):
            # get input
            v_s_s = v_support[nI:nI+seq_len, :]
            X_i[nI,:,:] = v_s_s[::-1,:]#v_support[nI:nI+seq_len, :]            
            # due to substraction of features for variation, output gets the 
            # feature one entry later
            O_i[nI,:,0] = r_support[nI]
            if calculate_roi:
                D_i[nI,:] = dt_support[nI,:]
                B_i[nI,:] = b_support[nI,:]
                A_i[nI,:] = a_support[nI,:]

        # normalize output
        O_i = O_i/stds_out[data.lookAheadIndex]
        # update counters
        offset = offset+batch
        # get decimal and binary outputs
        Y_i, y_dec = _build_bin_output(model, O_i, batch)
        # get samples per level
        for l in range(model.size_output_layer):
            totalSampsPerLevel[l] = totalSampsPerLevel[l]+np.sum(y_dec[:,-1,0]==l)
        # resize IO structures
        X.resize((pointer+batch, seq_len,features.shape[1]))
        Y.resize((pointer+batch, seq_len,model.commonY+model.size_output_layer))
        # update IO structures
        X[pointer:pointer+batch,:,:] = X_i
        Y[pointer:pointer+batch,:,:] = Y_i
        if calculate_roi:
            # resize
            D.resize((pointer+batch, 2))
            B.resize((pointer+batch, 2))
            A.resize((pointer+batch, 2))
            # update
            D[pointer:pointer+batch,:] = D_i
            B[pointer:pointer+batch,:] = B_i
            A[pointer:pointer+batch,:] = A_i
    #        save_as_matfile('X_h_n_'+str(int(s/2)),'X_h_n'+str(int(s/2)),X_i)
    #        save_as_matfile('O_h_n_'+str(int(s/2)),'O_h_n'+str(int(s/2)),O_i)

        # uodate pointer
        pointer += batch
    # end of for i in range(chunks):
    # update dictionary
    IO['X'] = X
    IO['Y'] = Y
    IO['pointer'] = pointer
    if calculate_roi:
        IO['D'] = D
        IO['B'] = B
        IO['A'] = A
    
    return IO, totalSampsPerLevel
    

In [4]:
# Snippet Build IO from var

# total number of possible channels
nChannels = int(nExS/mW)
# sequence length
seq_len = model.seq_len#int((data.lB-data.nEventsPerStat)/data.movingWindow)
# samples allocation per batch
aloc = 2**20
# extract means and stats
means_in = stats['means_in']
stds_in = stats['stds_in']
m_in = stats['m_in']
stds_out = stats['stds_out']
m_out = stats['m_out']
#print("m_in")
#print(m_in)
#print("m_out")
#print(m_out)
# add dateTimes, bids and asks if are included in file
all_info = 0
if calculate_roi:
    all_info = 1
    dts = symbols['DT']
    bids = symbols['B']
    asks = symbols['A']
        
    D = IO['D']
    B = IO['B']
    A = IO['A']

# extract IO structures
X = IO['X']
Y = IO['Y']
#I = IO['I']
pointer = IO['pointer']

feats_var_normed = np.minimum(np.maximum((features-means_in)/\
                     stds_in,-data.max_var),data.max_var)
# get some scalars
nSamps = feats_var_normed.shape[0]
samp_remaining = nSamps-nChannels-seq_len-1
print("samp_remaining")
print(samp_remaining)
chunks = int(np.ceil(samp_remaining/aloc))
# init counter of samps processed
offset = 0
# loop over chunks
for i in range(chunks):
    # this batch length
    batch = np.min([samp_remaining,aloc])
    print("batch")
    print(batch)
    # create support numpy vectors to speed up iterations
    v_support = feats_var_normed[offset:offset+batch+seq_len, :]
    # get init and end indexes for returns
    init_idx_rets = nChannels+offset+seq_len-1
    end_idx_rets = nChannels+offset+batch+2*seq_len-1
    r_support = returns[init_idx_rets:end_idx_rets, data.lookAheadIndex]
    print("returns.shape")
    print(returns.shape)
    print("nChannels+offset+2")
    print(nChannels+offset+2)
    print("nChannels+offset+batch+seq_len+2")
    print(nChannels+offset+batch+seq_len+2)
    print("r_support.shape")
    print(r_support.shape)
    print("nChannels+offset+batch+seq_len+2-(nChannels+offset+2)")
    print(nChannels+offset+batch+seq_len+2-(nChannels+offset+2))
    print("returns[-1,data.lookAheadIndex]")
    print(returns[-1,data.lookAheadIndex])
    print("r_support[-1]")
    print(r_support[-1])
    # we only take here the entry time index, and later at DTA building the 
    # exit time index is derived from the entry time and the number of events to
    # look ahead
    if calculate_roi:
        dt_support = dts[init_idx_rets:end_idx_rets, [0,data.lookAheadIndex+1]]
        b_support = bids[init_idx_rets:end_idx_rets, [0,data.lookAheadIndex+1]]
        a_support = asks[init_idx_rets:end_idx_rets, [0,data.lookAheadIndex+1]]
    # update remaining samps to proceed
    samp_remaining = samp_remaining-batch
    # init formatted input and output
    X_i = np.zeros((batch, seq_len, features.shape[1]))
    # real-valued output
    O_i = np.zeros((batch, seq_len, 1))    
    if calculate_roi:
        # last dimension is to incorporate in and out symbols
        D_i = np.chararray((batch, 2),itemsize=19)
        B_i = np.zeros((batch, 2))
        A_i = np.zeros((batch, 2))
    
    for nI in range(batch):
        # get input
        v_s_s = v_support[nI:nI+seq_len, :]
        X_i[nI,:,:] = v_s_s[::-1,:]#v_support[nI:nI+seq_len, :]            
        # due to substraction of features for variation, output gets the 
        # feature one entry later
        O_i[nI,:,0] = r_support[nI]
        if calculate_roi:
            D_i[nI,:] = dt_support[nI,:]
            B_i[nI,:] = b_support[nI,:]
            A_i[nI,:] = a_support[nI,:]
    
    # normalize output
    print("stds_out.shape")
    print(stds_out.shape)
    O_i = O_i/stds_out[data.lookAheadIndex]
    # update counters
    offset = offset+batch
    # get decimal and binary outputs
    Y_i, y_dec = _build_bin_output(model, O_i, batch)
    # get samples per level
    for l in range(model.size_output_layer):
        totalSampsPerLevel[l] = totalSampsPerLevel[l]+np.sum(y_dec[:,-1,0]==l)
    # resize IO structures
    X.resize((pointer+batch, seq_len,features.shape[1]))
    Y.resize((pointer+batch, seq_len,model.commonY+model.size_output_layer))
    # update IO structures
    X[pointer:pointer+batch,:,:] = X_i
    Y[pointer:pointer+batch,:,:] = Y_i
    if calculate_roi:
        # resize
        D.resize((pointer+batch, 2))
        B.resize((pointer+batch, 2))
        A.resize((pointer+batch, 2))
        # update
        D[pointer:pointer+batch,:] = D_i
        B[pointer:pointer+batch,:] = B_i
        A[pointer:pointer+batch,:] = A_i
#        save_as_matfile('X_h_n_'+str(int(s/2)),'X_h_n'+str(int(s/2)),X_i)
#        save_as_matfile('O_h_n_'+str(int(s/2)),'O_h_n'+str(int(s/2)),O_i)
        
    # uodate pointer
    pointer += batch
# end of for i in range(chunks):
# update dictionary
IO['X'] = X
IO['Y'] = Y
IO['pointer'] = pointer
if calculate_roi:
    IO['D'] = D
    IO['B'] = B
    IO['A'] = A
print("totalSampsPerLevel")
print(totalSampsPerLevel)

if if_build_IO:
    ass_IO_ass[ass_idx] = IO['pointer']
    #print("\tTime for "+thisAsset+":"+str(np.floor(time.time()-tic))+"s"+
    #          ". Total time:"+str(np.floor(time.time()-ticTotal))+"s")
# end of assets loop?
if if_build_IO:
    f_IO.attrs.create('ass_IO_ass', ass_IO_ass, dtype=int)
    f_IO.close()
else:
    # get ass_IO_ass from disk
    f_IO = h5py.File(filename_IO,'r')
    ass_IO_ass = f_IO.attrs.get("ass_IO_ass")
    f_IO.close()
m_t = IO['pointer']
print("m_t")
print(m_t)
print("IO['pointer']")
print(IO['pointer'])

m_in
42343
m_out
42333
samp_remaining
42319
batch
42319
returns.shape
(42333, 4)
nChannels+offset+2
12
nChannels+offset+batch+seq_len+2
42334
r_support.shape
(42321,)
nChannels+offset+batch+seq_len+2-(nChannels+offset+2)
42322
returns[-1,data.lookAheadIndex]
9.999999999998899e-05
r_support[-1]
9.999999999998899e-05
stds_out.shape
(4,)
totalSampsPerLevel
[  320.  6215. 29138.  6239.   407.]
m_t
42319
IO['pointer']
42319


In [5]:
f_IO.close()

In [6]:
O_i.shape

(42319, 3, 1)